# EE 564 Project 2

## Tesla Model S Motor Design


[Tesla Model S](http://www.teslamotors.com/models)

[RWD 85 Model](http://en.wikipedia.org/wiki/Tesla_Model_S#Powertrain)

* Max. Power: 360 hp (270 kW)
* Max. Torque: 441 Nm
* Top Speed: 225 km/h 

[Specs1](http://teslatap.com/undocumented/) and [Specs2](http://my.teslamotors.com/it_IT/forum/forums/tesla-model-s-motor-output)

**Useful Links:**

* [How does Tesla make their motors in the model S so small?](http://www.reddit.com/r/AskEngineers/comments/2mv2h5/how_does_tesla_make_their_motors_in_the_model_s/)

* [Tesla Motor Technology](http://my.teslamotors.com/roadster/technology/motor)

* [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf)

In [55]:
Pn = 270000
ntarget = 0.96
Vln=600
f1=60
p1=2
m=3

In [56]:
import math
KE=0.98-0.005*p1
cosphin=0.9
nn=0.96
Iln=Pn/(math.sqrt(3)*Vln*cosphin*nn)
Sgap=3*KE*Vln*Iln
print KE
print Sgap

0.97
525027.901044


In [57]:
C0=265e3
lambda1=1.1
Dis=math.pow((2*p1*p1*Sgap)/(3.14*lambda1*f1*C0),1/3)
print Dis

1.0


In [58]:
g=1.5e-3
tau=3.14*Dis/(2*p1)
li=lambda1*tau
print li

0.8635


In [59]:
print Vln

600


In [61]:
KD=0.63
Dout=Dis/KD
print Dout

1.5873015873


In [64]:
Bg=0.8
KB=1.1
alphai=0.69
Kw1=0.925
A1=C0/(KB*alphai*Kw1*3.14*Bg*2*p1)
print A1

37564.9434082


In [66]:
phi=alphai*tau*li*Bg
print phi

0.37417182


In [69]:
W1=KE*Vln/(4*KB*f1*Kw1*phi)
print W1

6.36951329818


In [70]:
a1=1
q1=6
Ns=2*p1*q1*m
ns=(2*m*a1*W1)/Ns
print ns

0.530792774848
